In [0]:
%%sh
# aws s3 ls s3://prod.appannie.mktint.data/oss/SEARCH_ADS_KEYWORD_SOV/routine/granularity=daily/date=2020-02-15/platform=1/app_type=SINGLE_APP/
# aws s3 ls s3://b2c-prod-data-pipeline-unified-aso/unified/app-tech.aso.sov-search-ads-keyword-fact.v1/fact/granularity=daily/date=2019-07-03/country_code=KR/
 
# aws s3 ls s3://b2c-prod-data-pipeline-unified-aso/unified/app-tech.aso.keyword-trafficshare.v1/
# aws s3 ls s3://b2c-prod-data-pipeline-unified-aso/unified/aso.keyword-trafficshare.v1/fact/granularity=daily/date=20÷19-01-11/dev÷ice_code=android-all/
 
# aws s3 ls s3://prod.appannie.aso.paid.product/aso/oss/traffic_share/version=1.0.0/range_type=DAY/
  #date=2020-02-22/device_code=android-all/ --recursive --human --summarize | tail -30

# aws s3 ls s3://b2c-prod-data-pipeline-unified-aso/unified/aso.sov-search-ads-keyword-fact.v1/fact/granularity=daily/
 aws s3 ls s3://b2c-prod-data-pipeline-unified-aso/unified/app-tech.market.aso-keyword-trafficshare.v1/fact/granularity=daily/





# aws s3 ls s3://b2c-prod-data-pipeline-unified-aso/unified/app-tech.aso.sov-search-ads-keyword-fact.v1/fact/granularity=daily/
# aws s3 ls s3://b2c-prod-data-pipeline-unified-aso/unified/app-tech.aso.sov-search-ads-keyword-fact.v1/fact/granularity=daily/date=2019-07-03/country_code=AU/





In [0]:

def get_compared_data(date, country):
    df_new = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-aso/unified/aso.sov-search-ads-keyword-fact.v1/fact/").where("granularity='daily'").filter("date='{}' and app_type='SINGLE_APP' and country_code='{}'".format(date,country))
    df_old = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-aso/unified/app-tech.aso.sov-search-ads-keyword-fact.v1/fact/granularity=daily/date={}/".format(date)).filter("app_type='SINGLE_APP' and country_code='{}'".format(country))

    columns_list = df_old.columns
    columns_list.remove("doc_id")
    list1= df_new.sample(False, 0.001, seed=0).limit(5).take(5)
    return list1, df_old

test_list, df_old = get_compared_data('2020-03-01','JP')



def compare(test_list, df_old):
    for x in range(5):
        list2 = df_old.filter("keyword_id='{}'".format(test_list[x]["keyword_id"])).collect()
        # print list2
        for column in columns_list:
            if test_list[x][column] != list2[0][column]:
                print 'not equal',column, test_list[x][column], list2[0][column]
    print 'pass'           

compare(test_list, df_old)

In [0]:

list_str = "','".join(map(str,diff))
# keyword_list_unified 
zero = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-aso/unified//fact/granularity=daily/date=2019-01-01/device_code=ios-phone/").filter("country_code='KW' and keyword_id in ('{}')".format(list_str)).collect()

# print keyword_list_unified[0]

# traffic_share_list = zero[0]["traffic_share_list"]

# for x in traffic_share_list:
#     print round(x, 6)==0

for record in zero:
    print record
    for traffic_share_score in record["traffic_share_list"]:
        # print traffic_share_score
        if round(traffic_share_score, 6)==0:
            pass
        else:
            print traffic_share_score

In [0]:

import pyspark.sql.functions as F
from pyspark.sql.functions import size
import psycopg2
import datetime

start = '2020-02-01'
end = '2020-02-29'
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range =  real_date2 - real_date1
dates = list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))


device_code_list=["ios-phone","ios-tablet","android-all"]



def get_unified_new_df(date,device):
    df=spark.read.parquet("s3://b2c-prod-data-pipeline-unified-aso/unified/aso.keyword-trafficshare.v1/fact/granularity=daily/date={}/device_code={}/".format(date,device)).cache()
    new_df = df.withColumn("traffic_share_list", F.expr("filter(traffic_share_list, x-> x>=0.0000005 )"))
    new_df = new_df.filter(F.size("traffic_share_list") >0)
    unified_new_df = new_df.groupBy("country_code").agg({"*":"count"}).collect()
    return unified_new_df
    
    

def get_data_in_citus(date,device):
    citus_dsn_ = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db="aa_mkt_db",
        user="citus_mkt_qa",
        host="internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com",
        password="rn*Wh%osCl2C",
        port=7432
        )
    )
    sql = "select country_code, count(*) from aso.aso_keyword_trafficshare_fact_v1  where date='{}' and device_code='{}' group by country_code ".format(date,device)
    db_data = query(citus_dsn_, sql)
    return db_data
# sql = "select * from aso.aso_keyword_trafficshare_fact_v1 where date='2019-01-01' and device_code='ios-phone'  and keyword_id = '1030'"
# sql = "select keyword_id from aso.aso_keyword_trafficshare_fact_v1 where date='2019-01-01' and device_code='ios-phone'"

# sql = "select count(*) from aso.aso_keyword_trafficshare_fact_v1  where date='2019-01-01' and device_code='ios-phone'  "
# sql = "select * from aso.aso_keyword_trafficshare_fact_v1 where date='2019-01-01' and device_code='ios-phone' and country_code='KW' limit 1"

def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result
    
# print unified_new_df[0]
# print unified_new_df[1]
# keyword_list_unified=new_df.filter("country_code='KW'").select("keyword_id").collect()




def compare(date,device, unified_new_df,test_result ):
    key = ["country_code", "count(1)"]
    unified_data_dict=dict()
    for data in unified_new_df:
        unified_data_dict[data[key[0]]]=data[key[1]]

    for x in range(0,len(unified_new_df)):
        if (test_result[x][1]-unified_data_dict[test_result[x][0]]) !=0:
            print '{}, {}, country code: {} , db {} , unified {}, diff db - unified : {}'.format(date,device, test_result[x][0] ,test_result[x][1], unified_data_dict[test_result[x][0]], test_result[x][1]-unified_data_dict[test_result[x][0]])




for d in dates:
    print d
    for device in device_code_list:
        compare(d,device, get_unified_new_df(d,device), get_data_in_citus(d,device))

    

In [0]:


import psycopg2
import datetime

def get_data_in_citus(date,device):
    citus_dsn_ = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db="aa_mkt_db",
        user="citus_mkt_qa",
        host="internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com",
        password="rn*Wh%osCl2C",
        port=7432
        )
    )
    sql = "select country_code, count(*) from aso.aso_keyword_trafficshare_fact_v1  where date='2019-01-01' and device_code='ios-phone' group by country_code "
    db_data = query(citus_dsn_, sql)
    return db_data
# sql = "select * from aso.aso_keyword_trafficshare_fact_v1 where date='2019-01-01' and device_code='ios-phone'  and keyword_id = '1030'"
# sql = "select keyword_id from aso.aso_keyword_trafficshare_fact_v1 where date='2019-01-01' and device_code='ios-phone'"

# sql = "select count(*) from aso.aso_keyword_trafficshare_fact_v1  where date='2019-01-01' and device_code='ios-phone'  "
# sql = "select * from aso.aso_keyword_trafficshare_fact_v1 where date='2019-01-01' and device_code='ios-phone' and country_code='KW' limit 1"

def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result
# query(citus_dsn_, sql)
# test_result = query(citus_dsn_, sql)



# print test_result

In [0]:

# print unified_new_df
test_result = keyword_id_list
key = ["country_code", "count(1)"]
unified_data_dict=dict()
for data in unified_new_df:
    unified_data_dict[data[key[0]]]=data[key[1]]

# print unified_data_dict
for x in range(0,len(unified_new_df)):
    print 'country code: {} , db {} , unified {}, diff db - unified : {}'.format(test_result[x][0] ,test_result[x][1], unified_data_dict[test_result[x][0]], test_result[x][1]-unified_data_dict[test_result[x][0]])

In [0]:

print keyword_id_list[0][0]
print keyword_list_unified[0]["keyword_id"]
unified_list = [x["keyword_id"] for x in keyword_list_unified ]
print unified_list[0]

db_list = [ x[0] for x in keyword_id_list ]
print db_list[0]

print 'new'
print len(keyword_id_list)
print len(unified_list)
# for keyword in keyword_id_list:
#     # print keyword
#     if keyword[0] not in unified_list:
        # print keyword[0]
diff = list(set(unified_list).difference(set(db_list)))
print diff


# list_str = "','".join(map(str,diff))
# zero = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-aso/unified/aso.keyword-trafficshare.v1/fact/granularity=daily/date=2019-01-01/device_code=ios-phone/").filter("country_code='KW' and keyword_id in ('{}')".format(list_str)).collect()
# for record in zero:
#     print record
#     for traffic_share_score in record["traffic_share_list"]:
#         # print traffic_share_score
#         if round(traffic_share_score, 6)==0:
#             pass
#         else:
#             print record


In [0]:

# unified_detail = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-aso/unified/aso.keyword-trafficshare.v1/fact/granularity=daily/date=2019-01-01/device_code=ios-phone/").filter("country_code='KW' and keyword_id='5145055'").collect()
# unified_cloumns=spark.read.parquet("s3://b2c-prod-data-pipeline-unified-aso/unified/aso.keyword-trafficshare.v1/fact/granularity=daily/date=2019-01-01/device_code=ios-phone/").filter("country_code='KW' and keyword_id='5145055'").columns


print detail
print unified_detail
print unified_cloumns.pop(0)

for x in range(0,len(unified_detail)):
    print unified_detail[unified_cloumns[x]]


In [0]:
%%sh
PGPASSWORD='rn*Wh%osCl2C' psql -h internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com  -U citus_mkt_qa -d aa_mkt_db -p 7432 << EOF 
set search_path=aso;
--select * from aso.aso_keyword_trafficshare_fact_v1  limit 1  ;

select count(*) from aso.aso_sov_search_ads_keyword_fact_v1 where country_code='US' and device_code='ios-phone' and granularity='daily' and date='2020-12-14';

EOF 



In [0]:

import datetime

start = '2019-09-14'
end = '2019-09-15'
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range =  real_date2 - real_date1
dates = list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))

count_list=[]
for x in dates:
    print x
    count_list.extend(spark.read.parquet("s3://b2c-prod-data-pipeline-unified-aso/unified/app-tech.aso.sov-search-ads-keyword-fact.v1/fact/granularity=daily/date={}/".format(x)).filter("app_type='SINGLE_APP' and device_code in ( 'ios-phone', 'android-all' ,'ios-tablet')").groupBy("_identifier").agg({"*":"count"}).orderBy("count(1)").collect())

In [0]:


import psycopg2
import datetime

citus_dsn_ = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db="aa_mkt_db",
        user="citus_mkt_qa",
        host="internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com",
        password="rn*Wh%osCl2C",
        port=7432
    )
)
# sql = "select * from aso.aso_keyword_trafficshare_fact_v1 limit 1;"
sql = "select count(distinct keyword_id) from aso.aso_keyword_trafficshare_fact_v1 where country_code='JP' and date='2017-01-01' and device_code='android-all' "

def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result
    
test_result = query(citus_dsn_, sql)

print test_result


In [0]:

from elasticsearch import Elasticsearch
import datetime
start = '2020-03-06'
end = '2020-03-09'
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range =  real_date2 - real_date1
dates = list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))
# print dates
def es_doc(date):
    # print 'review_id ', review_id
    es_connection = Elasticsearch(["http://internal-aa-prod-int-elasticsearch-v5-1589259492.us-east-1.elb.amazonaws.com:19200"],http_auth=("app_bdp_usage_qa","euIGw5c6cP3D"),port=19200)
    doc = es_connection.count(index="aso-kpi-sov_search_ads_keyword_fact_v1_daily*", body={"query":{"match":{"date":"{}".format(date)}}})
    return date, doc["count"]
for x in dates:
    print es_doc(str(x))



In [0]:

import datetime
start = '2020-03-06'
end = '2020-03-09'
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range =  real_date2 - real_date1
dates = list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))

for date in dates:
    print spark.read.parquet("s3://b2c-prod-data-pipeline-unified-aso/unified/app-tech.aso.sov-search-ads-keyword-fact.v1/fact/granularity=daily/date={}/".format(date)).filter("app_type='SINGLE_APP' and device_code in ( 'ios-phone', 'android-all' ,'ios-tablet')").groupBy("_identifier").agg({"*":"count"}).orderBy("count(1)").show()



In [0]:

import psycopg2
import datetime
import datetime
start = '2020-03-06'
end = '2020-03-09'
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range =  real_date2 - real_date1
dates = list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))

citus_dsn_ = (
    "dbname='{db}' user='{user}' password='{password}' "
    "host='{host}' port='{port}'".format(
        db="aa_mkt_db",
        user="citus_mkt_qa",
        host="internal-aa-citus-mkt-elb-1511434527.us-east-1.elb.amazonaws.com",
        password="rn*Wh%osCl2C",
        port=7432
    )
)
sql = "select count(*) from aso.aso_sov_search_ads_keyword_fact_v1 where date='{}' and granularity='daily'"

sql_list = [sql.format(x) for x in dates]
print sql_list
def query(dsn, sql):
    with psycopg2.connect(dsn) as conn:
        conn.autocommit = True
        with conn.cursor() as cur:
            cur.execute(sql)
            result = cur.fetchall()
            conn.commit()
    return result

result = []
for x in sql_list:
    result.extend(query(citus_dsn_, x))

for x in result:
    print x[0]